In [1]:
from os import listdir
from nltk.tokenize import word_tokenize
from string import punctuation
from konlpy.tag import Komoran
from collections import defaultdict
from math import log10, log2, sqrt
import re
import bad_at_io
import ngram
from string import punctuation
from re_DIY import re_DIY

### DTM

In [3]:
ma = Komoran()

In [4]:
tf = defaultdict(lambda: defaultdict(int))
max_tf = defaultdict(int)
df = defaultdict(int)

In [152]:
for section in listdir("scraping/Daum"):
    for f in bad_at_io.fileid("scraping/Daum/"+section+"/"):
        content = bad_at_io.txt_open(f)
        content = re_DIY(content)
        for token in word_tokenize(content):
            for _ in ma.pos(token):
                if _[1].startswith("N"):
                    tf[f]["/".join(_)] += 1
        for _ in tf[f].keys():
            df[_] += 1
        max_tf[f] = max(tf[f].values())

In [153]:
N = len(max_tf.keys())

In [154]:
idf = {key:log2(N/(value+1)) for key, value in df.items()}

In [155]:
doubleTF = defaultdict(lambda: defaultdict(int))
TFIDF = defaultdict(lambda: defaultdict(int))

In [156]:
for article, values in tf.items():
    for term, freq in values.items():
        doubleTF[article][term] = tf[article][term] / max_tf[article] 
        TFIDF[article][term] = 0.5 + (1-0.5) * (doubleTF[article][term] * idf[term])

### Query Feature

In [206]:
query = "돈까스"

In [207]:
tf_q = defaultdict(int)
df_q = defaultdict(int)
idf_q = defaultdict(int)

In [208]:
query = re_DIY(query)
for token in word_tokenize(query):
    for _ in ma.pos(token):
        tf_q["/".join(_)] +=1
        df_q["/".join(_)] = df["/".join(_)]
idf_q = {key:log2(N/(value+1)) for key, value in df_q.items()}

In [209]:
max_tf_q = max(tf_q.values())

In [210]:
doubleTF_q = defaultdict(int)
TFIDF_q = defaultdict(int)

In [211]:
for term, freq in  tf_q.items():
    doubleTF_q[term] = tf_q[term] / max_tf_q
    TFIDF_q[term] = 0.5 + (1-0.5) * (doubleTF_q[term] * idf_q[term])

In [212]:
TFIDF_q

defaultdict(int, {'돈까스/NA': 4.169925001442312})

### TDM

In [213]:
TFIDF_T = defaultdict(lambda: defaultdict(float))

In [214]:
for article, freq_dict in TFIDF.items():
    for term, freq in freq_dict.items():
        TFIDF_T[term][article] = TFIDF[article][term]

In [215]:
len(TFIDF.keys())

162

In [216]:
TFIDF_q

defaultdict(int, {'돈까스/NA': 4.169925001442312})

In [217]:
LENGTH = defaultdict(int)

In [218]:
for article, freq_dict in TFIDF.items():
    for term, freq in freq_dict.items():
        LENGTH[article] += freq**2

In [219]:
LENGTH = {key:sqrt(value) for key, value in LENGTH.items()}

In [220]:
COSINE = defaultdict(float)

In [221]:
for term, freq in TFIDF_q.items():
    if term in TFIDF_T.keys():
        for article, value in TFIDF_T[term].items():
            COSINE[article] += (TFIDF_T[term][article] * TFIDF_q[term]) / LENGTH[article]

### show top K

In [222]:
K = 5

In [223]:
search_result = sorted(COSINE.items(),key=lambda e: e[1], reverse = True)[:5]

In [224]:
for file, similarity in search_result:
    print(file.split("/")[-1])
    print("similarity is {0:.3f}%".format(similarity*100))
    print(" ".join(bad_at_io.txt_open(file).split()[:100]))
    print()